# スタッキング001

In [1]:
import torch
# GPUの名称、CUDA Compute Capabilityを取得
print(torch.cuda.get_device_name())

NVIDIA GeForce RTX 2080 Ti


In [2]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2
from FeatureEngineering import FeatureEngineering
from models import Model1Xgb, Model1NN, Model2Linear

%matplotlib inline
gc.enable()

print("OK\n")

OK



## 共通関数定義

In [3]:
### LightGBM
params_lgb = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}
### Xgboost
params_xgb = {
        'booster': 'gbtree'
        ,'objective': 'binary:logistic'
        ,'silent': 1
        ,'random_state': 17
        ,'eval_metric': 'auc'
        ,'eta': 0.1
        ,'max_depth': 8
        ,'subsample': 0.9
        ,'colsample_state': 71
        ,'colsample_bytree': 1.0
        ,'colsample_bylevel': 0.3
        ,'gamma': 0
        ,'lambda': 1
        ,'alpha': 0
        ,'min_child_weight': 1
        ,'gpu_id': 0
        ,'tree_method': 'gpu_hist'
}

In [4]:
# 学習データに対する「目的変数を知らない」予測値と、テストデータに対する予測値を返す関数
def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=4, shuffle=True, random_state=71)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        print(pred)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test

In [5]:
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}



def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=100
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

## インスタンス生成

In [6]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')
FeatureEngineering = FeatureEngineering('SK_ID_CURR')
FeatureEngineering2 = FeatureEngineering2('SK_ID_CURR')

## 特徴量生成1

In [7]:
### application
df = FeatureEngineering2.application()
df.head()

"Application_Train_Test" final shape: (356250, 238)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,APP_EXT_SOURCE_2*DAYS_EMPLOYED,APP_EXT_SOURCE_3*DAYS_EMPLOYED,APPS_GOODS_INCOME_RATIO,APPS_CNT_FAM_INCOME_RATIO,APPS_INCOME_EMPLOYED_RATIO,CREDIT_TO_GOODS_RATIO_2,APP_AMT_INCOME_TOTAL_12_AMT_ANNUITY_ratio,APP_INCOME_TO_EMPLOYED_RATIO,APP_DAYS_LAST_PHONE_CHANGE_DAYS_EMPLOYED_ratio,APP_DAYS_EMPLOYED_DAYS_BIRTH_diff
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,-167.498254,-88.782372,1.733333,202500.0,-317.896389,1.158397,-7825.5,-317.896389,1.780220,8824.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,-739.227981,NaN,4.183333,135000.0,-227.272727,1.145199,-13198.5,-227.272727,0.696970,15577.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,-125.080219,-164.152505,2.000000,67500.0,-300.000000,1.000000,-1125.0,-300.000000,3.622222,18821.0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,-1976.692297,NaN,2.200000,67500.0,-44.422507,1.052803,-18436.5,-44.422507,0.203027,15966.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,-980.478916,NaN,4.222222,121500.0,-39.993417,1.000000,-11740.5,-39.993417,0.364055,16894.0


In [8]:
### bureau_balance
df = df.merge(FeatureEngineering2.bureau_bb(), how='left', on='SK_ID_CURR')
df.head()

"Bureau/Bureau Balance" final shape: (305811, 200)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MIN,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MAX,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MEAN,CLOSED_DEBT_CREDIT_DIFF_MIN,CLOSED_DEBT_CREDIT_DIFF_MAX,CLOSED_DEBT_CREDIT_DIFF_MEAN,CLOSED_BUREAU_IS_DPD_MEAN,CLOSED_BUREAU_IS_DPD_SUM,CLOSED_BUREAU_IS_DPD_OVER120_MEAN,CLOSED_BUREAU_IS_DPD_OVER120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.0,0.0,0.0,0.0,135000.0,85245.0,0.0,0.0,0.0,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,22248.0,112500.0,69133.5,0.0,0.0,0.0,0.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,94500.0,94537.8,94518.9,0.0,0.0,0.0,0.0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,146250.0,146250.0,146250.0,0.0,0.0,0.0,0.0


In [9]:
### previos_application
df = df.merge(FeatureEngineering2.previous_application(), how='left', on='SK_ID_CURR')
df.head()

"Previous Applications" final shape: (338857, 321)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,REFUSED_PREV_GOODS_DIFF_MEAN,REFUSED_PREV_GOODS_DIFF_MAX,REFUSED_PREV_GOODS_DIFF_SUM,REFUSED_PREV_GOODS_APPL_RATIO_MEAN,REFUSED_PREV_GOODS_APPL_RATIO_MAX,REFUSED_DAYS_LAST_DUE_DIFF_MEAN,REFUSED_DAYS_LAST_DUE_DIFF_MAX,REFUSED_DAYS_LAST_DUE_DIFF_SUM,REFUSED_SIMPLE_INTERESTS_MEAN,REFUSED_SIMPLE_INTERESTS_MAX
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,1.0,1.0,NaN,NaN,0.0,0.015231,0.015231
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
### pos_cash
df = df.merge(FeatureEngineering2.pos_cash(), how='left', on='SK_ID_CURR')
df.head()

"Pos-Cash" balance final shape: (337252, 46)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_nan_MEAN,POS_COUNT,POS_LOAN_COMPLETED_MEAN,POS_COMPLETED_BEFORE_MEAN,POS_REMAINING_INSTALMENTS,POS_REMAINING_INSTALMENTS_RATIO,LATE_PAYMENT_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.000000,0.000000,0.0,0.0,19.0,0.000000,0.0,6.0,0.250000,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.000000,0.000000,0.0,0.0,28.0,0.250000,1.0,1.0,0.083333,0.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.000000,0.000000,0.0,0.0,4.0,0.250000,1.0,0.0,0.000000,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.047619,0.000000,0.0,0.0,21.0,0.458333,2.0,3.0,0.250000,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.000000,0.015152,0.0,0.0,66.0,0.209402,1.0,14.0,0.641667,0.0


In [11]:
### instllment_payment
df = df.merge(FeatureEngineering2.installment(), how='left', on='SK_ID_CURR')
df.head()

"Installments Payments" final shape: (339587, 85)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INS_D365PAYMENT_PERC_MAX,INS_D365DPD_DIFF_MEAN,INS_D365DPD_DIFF_MIN,INS_D365DPD_DIFF_MAX,INS_D365DPD_MEAN,INS_D365DPD_SUM,INS_D365INS_IS_DPD_UNDER_120_MEAN,INS_D365INS_IS_DPD_UNDER_120_SUM,INS_D365INS_IS_DPD_OVER_120_MEAN,INS_D365INS_IS_DPD_OVER_120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1.0,-17.363636,-24.0,-12.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,1.0,-6.181818,-23.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,1.0,-2.250000,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
### credit_card_balance
df = df.merge(FeatureEngineering2.credit_card(), how='left', on='SK_ID_CURR')
df.head()

"Credit Card Balance" final shape: (103558, 284)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INS_48M_SK_DPD_MAX,INS_48M_SK_DPD_SUM,INS_48M_DRAWING_LIMIT_RATIO_MIN,INS_48M_DRAWING_LIMIT_RATIO_MAX,INS_48M_LATE_PAYMENT_MEAN,INS_48M_LATE_PAYMENT_SUM,INS_48M_CARD_IS_DPD_UNDER_120_MEAN,INS_48M_CARD_IS_DPD_UNDER_120_SUM,INS_48M_CARD_IS_DPD_OVER_120_MEAN,INS_48M_CARD_IS_DPD_OVER_120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
### 変数選択001
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 1172 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 1126 features are remained after removing non-informative features
129 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 997 features are remained after removing features not interesting for LightGBM classifier
---=> 54 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 1051 features

---=> df final shape: (356250, 1051)  <=--- 



,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0,1,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,0,1,0,1,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,1,0,1,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,0,1,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,1,0,1,0,0,0,0


In [ ]:
### 変数選択002
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

In [14]:
# infをnanへ置換
df = df.replace([np.inf, -np.inf], np.nan)
df.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0,1,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,0,1,0,1,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,1,0,1,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,0,1,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,1,0,1,0,0,0,0


In [15]:
### 変数名から不正な文字を除去
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
print(df.shape)
df.head()

(356250, 1051)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0,1,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,0,1,0,1,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,1,0,1,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,0,1,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,1,0,1,0,0,0,0


In [16]:
### split train, and test datasets
app_train = df[df['TARGET'].notnull()]
app_test = df[df['TARGET'].isnull()]

print(app_test.shape)
app_test.head()

(48744, 1051)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
307506,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,0,1,1,0,0,0,0,0
307507,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,0,1,0,1,0,0,0,0
307508,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,0,1,0,1,1,0,1,0
307509,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,0,1,0,1,1,0,1,0
307510,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0,1,0,0,0,0,0


In [17]:
### indexを削除
app_train=app_train.reset_index()
app_test=app_test.reset_index()
app_test.head()

,level_0,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,307506,0,100001,NaN,1,0,0,0,135000.0,568800.0,...,0,1,0,1,1,0,0,0,0,0
1,307507,1,100005,NaN,0,0,0,0,99000.0,222768.0,...,0,1,0,1,0,1,0,0,0,0
2,307508,2,100013,NaN,0,1,0,0,202500.0,663264.0,...,0,1,0,1,0,1,1,0,1,0
3,307509,3,100028,NaN,1,0,0,2,315000.0,1575000.0,...,0,1,0,1,0,1,1,0,1,0
4,307510,4,100038,NaN,0,1,1,1,180000.0,625500.0,...,0,0,0,0,1,0,0,0,0,0


In [18]:
### indexを削除
app_train = app_train.drop(columns=['index'])
app_test = app_test.drop(columns=['index'])
app_test.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,307506,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,0,1,1,0,0,0,0,0
1,307507,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,0,1,0,1,0,0,0,0
2,307508,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,0,1,0,1,1,0,1,0
3,307509,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,0,1,0,1,1,0,1,0
4,307510,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0,1,0,0,0,0,0


### kerasのmodelを生成

In [19]:
# trainデータセット作成
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [20]:
# testデータセット作成
x_test = app_test.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [21]:
# infをnanへ置換
app_train = app_train.fillna(app_train.max() + (app_train.max() * 0.4))
app_train = app_train.replace([np.inf, -np.inf], np.nan)
app_train.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0,1,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,0,1,0,1,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,1,0,1,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,0,1,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,1,0,1,0,0,0,0


In [22]:
# 欠損値補完
app_test = app_test.replace([np.inf, -np.inf], np.nan)
app_test = app_test.fillna(app_test.max() + (app_test.max() * 0.4))
app_test.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,307506,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,0,1,1,0,0,0,0,0
1,307507,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,0,1,0,1,0,0,0,0
2,307508,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,0,1,0,1,1,0,1,0
3,307509,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,0,1,0,1,1,0,1,0
4,307510,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0,1,0,0,0,0,0


In [23]:
### 処理対象の列名を配列化
encoding_columns = list(app_test.select_dtypes(include='number').columns)
encoding_columns.remove('SK_ID_CURR')
encoding_columns.remove('TARGET')

In [24]:
### indexを設定
app_train = app_train.set_index('SK_ID_CURR')
app_test = app_test.set_index('SK_ID_CURR')
app_test.head()

,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,307506,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,0,1,0,1,1,0,0,0,0,0
100005,307507,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,0,1,0,1,0,1,0,0,0,0
100013,307508,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,0,1,0,1,0,1,1,0,1,0
100028,307509,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,0,1,0,1,0,1,1,0,1,0
100038,307510,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,1,0,0,0,0,0


In [25]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(app_train, app_test, num_cols)
        
app_test.head()

,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1.732056,NaN,0.720379,-0.717912,-0.664539,-0.577531,-0.307114,-0.075104,-0.448077,-0.232482,...,0.0,0.579736,-0.01036,0.580061,0.946577,-0.840113,-0.593453,-0.150513,-0.498057,0.0
100005,1.732068,NaN,-1.388158,-0.717912,-0.664539,-0.577531,-0.637959,-0.934826,-0.665962,-0.907160,...,0.0,0.579736,-0.01036,0.580061,-1.056438,1.190317,-0.593453,-0.150513,-0.498057,0.0
100013,1.732079,NaN,-1.388158,1.392928,-0.664539,-0.577531,0.313221,0.159594,2.913009,0.217303,...,0.0,0.579736,-0.01036,0.580061,-1.056438,1.190317,1.685055,-0.150513,2.007802,0.0
100028,1.732090,NaN,0.720379,-0.717912,-0.664539,2.192099,1.347112,2.424818,1.495373,2.578675,...,0.0,0.579736,-0.01036,0.580061,-1.056438,1.190317,1.685055,-0.150513,2.007802,0.0
100038,1.732102,NaN,-1.388158,1.392928,1.504802,0.807284,0.106443,0.065768,0.337723,0.206059,...,0.0,-1.724923,-0.01036,-1.723955,0.946577,-0.840113,-0.593453,-0.150513,-0.498057,0.0


In [26]:
### indexを削除
app_train = app_train.reset_index()
app_test = app_test.reset_index()
app_test.head()

,SK_ID_CURR,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,100001,1.732056,NaN,0.720379,-0.717912,-0.664539,-0.577531,-0.307114,-0.075104,-0.448077,...,0.0,0.579736,-0.01036,0.580061,0.946577,-0.840113,-0.593453,-0.150513,-0.498057,0.0
1,100005,1.732068,NaN,-1.388158,-0.717912,-0.664539,-0.577531,-0.637959,-0.934826,-0.665962,...,0.0,0.579736,-0.01036,0.580061,-1.056438,1.190317,-0.593453,-0.150513,-0.498057,0.0
2,100013,1.732079,NaN,-1.388158,1.392928,-0.664539,-0.577531,0.313221,0.159594,2.913009,...,0.0,0.579736,-0.01036,0.580061,-1.056438,1.190317,1.685055,-0.150513,2.007802,0.0
3,100028,1.732090,NaN,0.720379,-0.717912,-0.664539,2.192099,1.347112,2.424818,1.495373,...,0.0,0.579736,-0.01036,0.580061,-1.056438,1.190317,1.685055,-0.150513,2.007802,0.0
4,100038,1.732102,NaN,-1.388158,1.392928,1.504802,0.807284,0.106443,0.065768,0.337723,...,0.0,-1.724923,-0.01036,-1.723955,0.946577,-0.840113,-0.593453,-0.150513,-0.498057,0.0


In [27]:
app_train = FeatureEngineering2.reduce_mem_usage(app_train)
app_train.head()

,SK_ID_CURR,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,100002,-1.732422,1.0,-1.387695,-0.717773,-0.664551,-0.577637,0.313232,-0.478027,-0.165405,...,0.0,0.579590,-0.010361,0.580078,0.946777,-0.840332,-0.593262,-0.150513,-0.498047,0.0
1,100003,-1.732422,0.0,0.720215,-0.717773,1.504883,-0.577637,0.933594,1.725586,0.585938,...,0.0,0.579590,-0.010361,0.580078,-1.056641,1.190430,-0.593262,-0.150513,-0.498047,0.0
2,100004,-1.732422,0.0,-1.387695,1.392578,-0.664551,-0.577637,-0.927246,-1.153320,-1.391602,...,0.0,0.579590,-0.010361,0.580078,-1.056641,1.190430,-0.593262,-0.150513,-0.498047,0.0
3,100006,-1.732422,0.0,0.720215,-0.717773,-0.664551,-0.577637,-0.307129,-0.711426,0.175171,...,0.0,-1.724609,-0.010361,-1.723633,-1.056641,1.190430,1.684570,-0.150513,-0.498047,0.0
4,100007,-1.732422,0.0,-1.387695,-0.717773,-0.664551,-0.577637,-0.431152,-0.213745,-0.358887,...,0.0,0.579590,-0.010361,0.580078,-1.056641,1.190430,-0.593262,-0.150513,-0.498047,0.0


In [28]:
app_test = FeatureEngineering2.reduce_mem_usage(app_test)
app_test.head()

,SK_ID_CURR,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,100001,1.732422,NaN,0.720215,-0.717773,-0.664551,-0.577637,-0.307129,-0.075134,-0.447998,...,0.0,0.579590,-0.010361,0.580078,0.946777,-0.840332,-0.593262,-0.150513,-0.498047,0.0
1,100005,1.732422,NaN,-1.387695,-0.717773,-0.664551,-0.577637,-0.638184,-0.935059,-0.666016,...,0.0,0.579590,-0.010361,0.580078,-1.056641,1.190430,-0.593262,-0.150513,-0.498047,0.0
2,100013,1.732422,NaN,-1.387695,1.392578,-0.664551,-0.577637,0.313232,0.159546,2.912109,...,0.0,0.579590,-0.010361,0.580078,-1.056641,1.190430,1.684570,-0.150513,2.007812,0.0
3,100028,1.732422,NaN,0.720215,-0.717773,-0.664551,2.191406,1.346680,2.425781,1.495117,...,0.0,0.579590,-0.010361,0.580078,-1.056641,1.190430,1.684570,-0.150513,2.007812,0.0
4,100038,1.732422,NaN,-1.387695,1.392578,1.504883,0.807129,0.106445,0.065796,0.337646,...,0.0,-1.724609,-0.010361,-1.723633,0.946777,-0.840332,-0.593262,-0.150513,-0.498047,0.0


In [29]:
# trainデータセット作成
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [30]:
# testデータセット作成
x_test = app_test.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [31]:
### training keras
# 1層目のモデル
# pred_train_1a, pred_train_1bは、学習データのクロスバリデーションでの予測値
# pred_test_1a, pred_test_1bは、テストデータの予測値
model_1b = Model1NN()
pred_train_1b, pred_test_1b = predict_cv(model_1b, x_train, y_train, x_test)

Epoch 1/10
1802/1802 [==============================] - 7s 3ms/step - loss: 0.2597 - accuracy: 0.9187 - val_loss: 0.2450 - val_accuracy: 0.9198
Epoch 2/10
1802/1802 [==============================] - 5s 3ms/step - loss: 0.2494 - accuracy: 0.9194 - val_loss: 0.2501 - val_accuracy: 0.9196
Epoch 3/10
1802/1802 [==============================] - 5s 3ms/step - loss: 0.2475 - accuracy: 0.9194 - val_loss: 0.2441 - val_accuracy: 0.9197
Epoch 4/10
1802/1802 [==============================] - 5s 3ms/step - loss: 0.2462 - accuracy: 0.9196 - val_loss: 0.2435 - val_accuracy: 0.9198
Epoch 5/10
1802/1802 [==============================] - 5s 3ms/step - loss: 0.2448 - accuracy: 0.9193 - val_loss: 0.2428 - val_accuracy: 0.9198
Epoch 6/10
1802/1802 [==============================] - 5s 3ms/step - loss: 0.2438 - accuracy: 0.9198 - val_loss: 0.2417 - val_accuracy: 0.9197
Epoch 7/10
1802/1802 [==============================] - 5s 3ms/step - loss: 0.2428 - accuracy: 0.9197 - val_loss: 0.2429 - val_accuracy:

In [95]:
pred_test_1b

array([[0.05105488],
       [0.16606653],
       [0.04389864],
       ...,
       [0.02031465],
       [0.07570788],
       [0.16833086]], dtype=float32)

In [109]:
pred_train_1b

array([[0.51685584],
       [0.0106446 ],
       [0.05407687],
       ...,
       [0.01276717],
       [0.11682555],
       [0.10738641]], dtype=float32)

In [138]:
print(type(sk_id_curr))

<class 'list'>


In [143]:
pred_1b = []
for a in np.array(pred_train_1b):
    pred_1b.append(a[0])

#pred_1b = np.array(pred_1b)
len(pred_1b)

307506

In [145]:
print(type(pred_1b))

<class 'list'>


In [147]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = app_train['SK_ID_CURR'].copy().to_list()
train_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'pred_1b': pred_1b})
print(train_x_2.shape)
train_x_2.head()

(307506, 2)


,SK_ID_CURR,pred_1b
0,100002,0.516856
1,100003,0.010645
2,100004,0.054077
3,100006,0.059492
4,100007,0.076165


In [148]:
train_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/pred_train_1b.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [149]:
pred_1b = []
for a in np.array(pred_test_1b):
    pred_1b.append(a[0])

#pred_1b = np.array(pred_1b)
len(pred_1b)

48744

In [151]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = app_test['SK_ID_CURR'].copy().to_list()
test_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'pred_1b': pred_1b})
print(test_x_2.shape)
test_x_2.head()

(48744, 2)


,SK_ID_CURR,pred_1b
0,100001,0.051055
1,100005,0.166067
2,100013,0.043899
3,100028,0.056542
4,100038,0.183168


In [152]:
test_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/pred_test_1b.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [ ]:
del df
del x_train
del y_train
del x_test
del app_train
del app_test
del model_1b
gc.collect()

## 特徴量生成2